[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1vILZZS2X-p9TxX8GoxpJ4vx2AFXoyn33#scrollTo=FmQxPYuNsGud)

# **FlowerLover Recommendation System**

## 01. Enviroment Setup


### 📦 Installing and Importing Neccesary Libraries

In [22]:
!pip install neo4j tensorflow keras &>/dev/null

In [23]:
from neo4j import GraphDatabase
import tensorflow as tf
from keras.models import load_model
import pandas as pd
import numpy as np

In [24]:
# Suppress Warnings
import warnings
warnings.filterwarnings("ignore")

### 🌐 Setting Up and Connecting to Neo4j using Neo4j Aura

🔐 Reuse credentials from the previous notebook `FlowerLover Knowledge Graph Creation using Neo4j.ipynb` for the Neo4j instance


In [25]:
NEO4J_URI = "neo4j+s://<your_neo4j_uri>"
NEO4J_USERNAME = "<your_username>"
NEO4J_PASSWORD = "<your_password>"

💐 The `FlowerRecommendationSystem` class
interacts with a Neo4j database to provide flower recommendations through efficient query execution and resource management


In [26]:
class FlowerRecommendationSystem:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def query(self, query, parameters=None):
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return [record.data() for record in result]

💡 Instantiating the recommendation system

In [27]:
recommender = FlowerRecommendationSystem(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)
print("Connected to Neo4j Aura instance.")

Connected to Neo4j Aura instance.


## 02. Personalized Recommendation Strategies

### Cold-Start Recommendations for New Users 🆕👶

When a new user joins a system, there's often not enough data about their preferences to make personalized recommendations.
This is known as the **cold-start problem**. 😕

In these cases, traditional recommendation methods like collaborative filtering struggle since they rely on past user interactions. Instead, recommendations are made based on **flowers popularity** 🔥 or **their high ratings from other users** 🌟.

As the user interacts more with the system, the recommendations become more tailored to their unique tastes. 💬

This approach ensures that even without personal data, new users still get helpful, high-quality suggestions. 🌸

In [28]:
def cold_start_recommendations():
  query = """
  MATCH (f:Species)<-[r:RATED]-(u:User)
  RETURN f.name AS flower, AVG(r.rating) AS avg_rating, COUNT(r) AS rating_count
  ORDER BY avg_rating DESC, rating_count DESC
  LIMIT 10
  """

  return recommender.query(query)

In [29]:
cold_start = cold_start_recommendations()

print("Cold Start Recommendations:\n")
pd.DataFrame(cold_start).head(10)

Cold Start Recommendations:



,flower,avg_rating,rating_count
0,hibiscus,5.000000,4
1,red ginger,5.000000,1
2,blackberry lily,5.000000,1
3,king protea,5.000000,1
4,californian poppy,5.000000,1
5,cyclamen,4.714286,7
6,daffodil,4.500000,6
7,moon orchid,4.500000,6
8,geranium,4.500000,2
9,tiger lily,4.400000,5


### Collaborative Filtering: Recommendations from Similar Users 🤝👥

**Collaborative Filtering (CF)** is a widely used technique that recommends items based on the preferences of similar users. If two users rate the same items similarly, it's assumed that they have similar tastes. 🪻

In real-world scenarios, when enough data is available, recommendations are based on the preferences of similar users. If data is limited (e.g., new users or items), it defaults to recommending popular flowers🌷⭐, based on overall ratings.

In [30]:
def collaborative_filtering_recommendations(user_id):
    query = """
    OPTIONAL MATCH (u1:User {id: $user_id})-[:RATED]->(f:Species)<-[:RATED]-(u2:User)
    WHERE u1 <> u2
    WITH f, COUNT(u2) AS similar_user_count
    ORDER BY similar_user_count DESC
    LIMIT 10
    RETURN f.name AS flower, similar_user_count AS score, 'Collaborative' AS recommendation_type

    UNION

    // Popular flowers for cold-start users
    MATCH (f:Species)<-[r:RATED]-(:User)
    RETURN f.name AS flower, AVG(r.rating) AS score, 'Popular' AS recommendation_type
    ORDER BY score DESC
    LIMIT 10
    """
    return recommender.query(query, {"user_id": user_id})


In [31]:
user_id = "U027"  # User ID in The KG is between "U001" and "U100"
collaborative = collaborative_filtering_recommendations(user_id)

print("Collaborative Filtering Recommendations:\n")
pd.DataFrame(collaborative).head(10)

Collaborative Filtering Recommendations:



,flower,score,recommendation_type
0,moon orchid,5.000000,Collaborative
1,grape hyacinth,5.000000,Collaborative
2,ball moss,2.000000,Collaborative
3,alpine sea holly,1.000000,Collaborative
4,red ginger,5.000000,Popular
5,hibiscus,5.000000,Popular
6,blackberry lily,5.000000,Popular
7,king protea,5.000000,Popular
8,californian poppy,5.000000,Popular
9,cyclamen,4.714286,Popular


**Collaborative Filtering with Jaccard Similarity** 🔍📊

Jaccard Similarity measures how similar two users are based on the flowers they've rated. It’s calculated as the size of the intersection ∩ of their rated flowers divided by the size of their union ∪.

In [32]:
def jaccard_similarity_recommendations (user_id):

    query = """
    MATCH (u1:User {id: $user_id})-[:RATED]->(f:Species)<-[:RATED]-(u2:User)
    WHERE u1 <> u2
    WITH u1, u2, f, COLLECT(DISTINCT f) AS u1_flowers,
         [(u2)-[:RATED]->(f2:Species) | f2] AS u2_flowers
    WITH u1, u2, u1_flowers, u2_flowers, f,
         apoc.coll.intersection(u1_flowers, u2_flowers) AS common_flowers,
         apoc.coll.union(u1_flowers, u2_flowers) AS all_flowers
    WITH u2, f, SIZE(common_flowers) * 1.0 / SIZE(all_flowers) AS jaccard_similarity
    WHERE jaccard_similarity > 0.1  // Filter for meaningful similarity
    MATCH (u1)-[:RATED]->(rated:Species) // Get all the flowers that user u1 has rated
    WITH u1, u2, f, jaccard_similarity, COLLECT(DISTINCT rated) AS rated_flowers
    WHERE NOT f IN rated_flowers // Exclude already rated flowers by user u1
    RETURN f.name AS flower, AVG(jaccard_similarity) AS avg_similarity
    ORDER BY avg_similarity DESC
    LIMIT 10
    """

    return recommender.query(query, {"user_id": user_id})


In [33]:
jaccard_recommendations = jaccard_similarity_recommendations(user_id)

print("Jaccard Similarity Recommendations:\n")
pd.DataFrame(jaccard_recommendations).head(10)

Jaccard Similarity Recommendations:



,flower,avg_similarity
0,moon orchid,0.25
1,grape hyacinth,0.25
2,alpine sea holly,0.25
3,ball moss,0.25


### Content-Based Filtering: Recommendations Based on User Preferences ❤️👤

Content-based filtering suggests flowers based on attributes the user has shown interest in, like preferred colors 🎨 or gardening experience 🌱.

By querying the knowledge graph, the system identifies flowers with similar traits, ensuring recommendations are closely aligned with the user's tastes and needs.

In [34]:
def content_based_recommendations(user_id):
    query = """
    MATCH (u:User {id: $user_id})-[:PREFERS_COLOR]->(c:Color),
          (f:Species)-[:HAS_COLOR]->(c)
    OPTIONAL MATCH (f)-[:REQUIRES_EXPERIENCE]->(:ExperienceLevel)<-[:HAS_EXPERIENCE]-(u)
    RETURN DISTINCT f.name AS flower, COUNT(*) AS rank
    ORDER BY rank DESC
    LIMIT 10
    """
    return recommender.query(query, {"user_id": user_id})

In [35]:
content_based = content_based_recommendations(user_id)

print("Content-Based Recommendations:\n")
pd.DataFrame(content_based).head(10)

Content-Based Recommendations:



,flower,rank
0,columbine,2
1,balloon flower,2
2,morning glory,2
3,love in the mist,2
4,windflower,2
5,monkshood,1
6,alpine sea holly,1
7,stemless gentian,1
8,pincushion flower,1
9,globe thistle,1


### Hybrid Recommendation with Weighted Score 🔗

The **Hybrid Recommendation** combines Content-Based Filtering and Collaborative Filtering for more accurate and personalized suggestions.

It assigns weights to each technique’s contribution—calculating a weighted sum of content relevance and collaborative scores.

This approach helps overcome the limitations of each method, such as the cold-start problem in collaborative filtering and the lack of diversity in content-based filtering.

In [36]:
def hybrid_recommendations(user_id, content_weight=0.6, collaborative_weight=0.4):

  query = f"""
  MATCH (u:User {{id: $user_id}})-[:PREFERS_COLOR]->(c:Color),
        (f:Species)-[:HAS_COLOR]->(c)
  OPTIONAL MATCH (u)-[r:RATED]->(f)
  OPTIONAL MATCH (:User)-[r2:RATED]->(f)
  RETURN f.name AS flower,COUNT(DISTINCT c) * {content_weight} + COUNT(DISTINCT r2) * {collaborative_weight} AS score
  ORDER BY score DESC
  LIMIT 10
  """

  return recommender.query(query, {"user_id": user_id})


In [37]:
hybrid = hybrid_recommendations(user_id, content_weight=0.7, collaborative_weight=0.3)

print("Hybrid Recommendations:\n")
pd.DataFrame(hybrid).head(10)

Hybrid Recommendations:



,flower,score
0,balloon flower,4.1
1,poinsettia,4.0
2,azalea,4.0
3,japanese anemone,4.0
4,primula,3.4
5,anthurium,3.4
6,love in the mist,3.2
7,monkshood,3.1
8,passion flower,3.1
9,pincushion flower,3.1
